## COVID19 Forecasting Project

**ECE 685:** Deep Learning <br>
**Student:** Guillem Amat, Sebastian Soriano

<br>

#### Loading Packages

In [163]:
# Normal Packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import pdb
import csv
import os

In [164]:
# Torch Packages
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

<br>

#### Loading Data

In [165]:
import warnings
warnings.filterwarnings("ignore")

In [166]:
PATH = r'C:\Users\guill\Desktop\Final Project'
os.chdir(PATH)

In [167]:
data = pd.read_csv(os.path.join(os.getcwd(), 'Data', 'week_5', 'train.csv'))

<br>

**Creating Key for Location**

In [168]:
data.head()

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue
0,1,NaN,NaN,Afghanistan,27657145,0.058359,1/23/2020,ConfirmedCases,0
1,2,NaN,NaN,Afghanistan,27657145,0.583587,1/23/2020,Fatalities,0
2,3,NaN,NaN,Afghanistan,27657145,0.058359,1/24/2020,ConfirmedCases,0
3,4,NaN,NaN,Afghanistan,27657145,0.583587,1/24/2020,Fatalities,0
4,5,NaN,NaN,Afghanistan,27657145,0.058359,1/25/2020,ConfirmedCases,0


In [169]:
data['County']         = data['County'].fillna('')
data['Province_State'] = data['Province_State'].fillna('')

In [170]:
data['location'] = data['Country_Region'] + ' ' + data['Province_State'] + ' ' + data['County']

In [171]:
population = data[['location', 'Population']].drop_duplicates()

<br>

**Filtering Country**

In [876]:
#!pip install us
#!pip install countrygroups

In [172]:
from countrygroups import EUROPEAN_UNION
#from us import states

In [173]:
european = EUROPEAN_UNION.names
#american = [str(state) for state in states.STATES]

In [174]:
# Operations to subset for european countries
data = data[data['Country_Region'].isin(european)]
data = data.loc[lambda x: x.Province_State == '']

In [82]:
#data = data[data['Country_Region']=='Spain']

<br>

**Pivoting Data**

In [175]:
data = data.pivot(index=['location', 'Date'], columns='Target', values='TargetValue')

In [176]:
data = data.reset_index()

<br>

**Joining Latitude and Longitude**

In [177]:
coordinates = pd.read_csv(os.path.join(os.getcwd(), 'Data', 'other', 'coordinates.csv'))

In [178]:
coordinates.head()

,Location,Longitude,Latitude
0,Afghanistan,66.238514,33.768006
1,Albania,19.999962,41.000028
2,Algeria,2.999983,28.000027
3,Andorra,1.573203,42.540717
4,Angola,17.569124,-11.877577


In [179]:
df = data.merge(coordinates, how='inner', left_on='location', right_on='Location')
data = df.merge(population, how='left', left_on='location', right_on='location')

In [180]:
data = data.drop(['Location', 'location', 'Date'], axis = 1)

In [181]:
data.head()

,ConfirmedCases,Fatalities,Longitude,Latitude,Population
0,0,0,13.199959,47.200034,8725931
1,0,0,13.199959,47.200034,8725931
2,0,0,13.199959,47.200034,8725931
3,0,0,13.199959,47.200034,8725931
4,0,0,13.199959,47.200034,8725931


<br>

**Scaling Variables**

In [1254]:
# Scaling Variables
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(data)
scaled_data = scaler.transform(data)

<br>

**Creating Numpy Array**

In [28]:
dataset = np.array(scaled_data).reshape(-1, 140, 5)
train = dataset[:, :95, :]
test  = dataset[:, 95:, :]

<br>

**Creating Sequences**

In [ ]:
def create_sequences(data, seq_length, n_features=2):
    
    X = np.zeros((data.shape[0], data.shape[1] -seq_length -1, seq_length, data.shape[2]))
    Y = np.zeros((data.shape[0], data.shape[1] -seq_length -1, n_features))
    
    for batch in range(X.shape[0]):
        
        xs = []; ys = []
        country = data[batch]
        
        for i in range(len(country) -seq_length -1):
            x = data[batch][i:(i+seq_length)]
            y = data[batch][i+seq_length]
            xs.append(x)
            ys.append(y)
            
        #pdb.set_trace()
        X[batch, :, :, :] = np.array(xs)
        Y[batch, :,  :]   = np.array(ys)[:, :2]
        
    return X, Y

In [ ]:
seq_length = 25
X_train, y_train = create_sequences(train, seq_length=seq_length, n_features=2)
X_test,  y_test  = create_sequences(train, seq_length=seq_length, n_features=2)

In [ ]:
X_train, y_train = torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()
X_test, y_test = torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()

In [ ]:
trainset = torch.utils.data.TensorDataset(X_train, y_train)
testset  = torch.utils.data.TensorDataset(X_test, y_test)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)
testloader  = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)

<br>

### Modeling

In [ ]:
torch.set_default_tensor_type(torch.FloatTensor)

<br>

#### Artificial Neural Network

<br>

#### Convolutional Neural Network

<br>

#### Recurrent Neural Network

<br>

#### Gated Recurrent Unit

In [ ]:
class GRU(nn.Module):
    def __init__(self, n_input, n_hidden, n_output, n_layers=2, )

<br>

#### Long-Short Term Memory

In [ ]:
class LSTM(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers):
        super().__init__()
        self.n_hidden = n_hidden
        self.seq_len  = seq_len
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_size=n_features,
                            hidden_size=n_hidden,
                            num_layers=n_layers,
                            dropout=0.01)
        
        self.linear = nn.Linear(in_features=n_hidden, out_features=2)
     
    def reset_hidden_state(self):
        self.hidden = (torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
                       torch.zeros(self.n_layers, self.seq_len, self.n_hidden))
            
    def forward(self, sequences):
        #pdb.set_trace()
        lstm_out, self.hidden = self.lstm(sequences.view(len(sequences), self.seq_len, -1),
                                          self.hidden)
            
        last_time_step = lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred  

In [229]:
criterion = torch.nn.MSELoss(reduction='sum')
model     = LSTMCoNet(n_features=5, n_hidden=512, seq_len=25, n_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

In [230]:
def train_model(model: torch.nn.Module, train: torch.utils.data.DataLoader, test: torch.utils.data.DataLoader, epochs: int):
    
    #Initializing Variables to store information
    min_loss   = float('inf')
    train_loss = 0
    test_loss  = 0
    train_history = []
    test_history  = []
    
    for epoch in range(epochs):
        
        train_loss = 0
        test_loss  = 0
    
        model.train()
        for _, data in enumerate(train):
        
            model.reset_hidden_state()
            optimizer.zero_grad()
            
            X_train, y_train = data
            X_train, y_train = Variable(torch.squeeze(X_train)), Variable(torch.squeeze(y_train))
            y_hat = model(X_train)
            
            loss  = criterion(y_hat, y_train)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() 
            
        train_history.append(train_loss)
        
        model.eval()
        for _, data in enumerate(test):
            
            X_test, y_test = data
            X_test, y_test = torch.squeeze(X_test), torch.squeeze(y_test)
            y_hat = model(X_test)
            
            test_loss += criterion(y_hat, y_test).item() 
       
        test_history.append(test_loss)
        
        train_loss     = str(np.mean(train_loss))[:4]
        test_loss      = test_loss/len(test.dataset)
        
        if test_loss < min_loss:
            min_test_loss = test_loss
            
            #Save Model
            PATH = os.path.join(os.getcwd(), 'Models', 'LSTM_model')
            torch.save(model, PATH)
            
            #Save State Dictionary
            PATH = os.path.join(os.getcwd(), 'Models', 'LSTM_state_dictionary')
            torch.save(model.state_dict(), PATH)
        
        if epoch == 0:
            string = f'''| Epoch: {epoch + 1}   | Train Loss: {train_loss} | Test Loss: {str(test_loss)[:4]} |'''
            print('-'*len(string)); print(string); print('-'*len(string))
    
        elif ((epoch + 1) % 10 == 0) & (epoch < 99):
            string = f'''| Epoch: {epoch + 1}  | Train Loss: {train_loss} | Test Loss: {str(test_loss)[:4]} |'''
            print(string); print('-'*len(string))
            
        elif ((epoch + 1) % 10 == 0) & (epoch >= 100):
            string = f'''| Epoch: {epoch + 1} | Train Loss: {train_loss} | Test Loss: {str(test_loss)[:4]} |'''
            print(string); print('-'*len(string))
        
    return model.eval(), history

In [ ]:
lstm, history = train_model(model, trainloader, testloader, 100)

---------------------------------------------------
| Epoch: 1   | Train Loss: 147. | Test Loss: 0.48 |
---------------------------------------------------
| Epoch: 10  | Train Loss: 8.39 | Test Loss: 0.19 |
---------------------------------------------------
| Epoch: 20  | Train Loss: 5.85 | Test Loss: 0.19 |
---------------------------------------------------
